# Project Part 1
## Imports and Data Wrangling

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.optimize import brentq

ois_rates = pd.read_excel("IR Data.xlsx", sheet_name = 'OIS', usecols = ["Tenor", "Product", "Rate"])

In [2]:
#tenors = ['6m', '1y', '2y', '3y', '4y', '5y', '7y', '10y', '15y', '20y', '30y']
tenors = ois_rates["Tenor"]

# Define a function to convert tenor strings to years
def tenor_to_years(tenor):
    if 'm' in tenor:
        return int(tenor.replace('m', '')) / 12
    elif 'y' in tenor:
        return int(tenor.replace('y', ''))
    else:
        return None

# Convert each tenor to years
ois_rates["years"] = [tenor_to_years(tenor) for tenor in tenors]

# Add place holder for Discount Factors and cumsum of discount factors
ois_rates[['cumsum_df','df']] = np.nan
ois_rates

,Tenor,Product,Rate,years,cumsum_df,df
0,6m,OIS,0.00250,0.5,NaN,NaN
1,1y,OIS,0.00300,1.0,NaN,NaN
2,2y,OIS,0.00325,2.0,NaN,NaN
3,3y,OIS,0.00335,3.0,NaN,NaN
4,4y,OIS,0.00350,4.0,NaN,NaN
5,5y,OIS,0.00360,5.0,NaN,NaN
6,7y,OIS,0.00400,7.0,NaN,NaN
7,10y,OIS,0.00450,10.0,NaN,NaN
8,15y,OIS,0.00500,15.0,NaN,NaN
9,20y,OIS,0.00525,20.0,NaN,NaN


## Solve for year 1 Discount Factor
Use the swap formula where we consider a par swap as the following:
$$
\sum_{i=1}^{n} D(0, T_i) \times \Delta_{i-1} \times L(T_{i-1}, T_i) = 1 - D(0, T_n) \\
\sum_{i=1}^{n} D(0, T_i) \times \Delta_{i-1} \times L(T_{i-1}, T_i) - (1 - D(0, T_n))= 0 \tag{1}
$$
Equation (1) can be a function of the unknown discount factor and can be used as objective function. After using Solver to find the root of equation (1), this root is the Discount Factor(df) for that year.

In [3]:
def par_swap_1y(df, ois_rate):
    """
    Return PV of payer swap i.e. PV of fixed leg minus PV of floating leg 
    FOR FIRST YEAR ONLY
    paramters:
        df: discount factor a year
        ois_rate: rate for the same year
    Assume annual payout since day count fraction is not included
    """
    return ois_rate*df - (1 - df)
    
guess = 0.99

ois_rates.at[1, 'df'] = fsolve(par_swap_1y, guess, args=(ois_rates.at[1, 'Rate']))[0]
ois_rates.at[1, 'cumsum_df'] = ois_rates.at[1, 'df']
ois_rates

,Tenor,Product,Rate,years,cumsum_df,df
0,6m,OIS,0.00250,0.5,NaN,NaN
1,1y,OIS,0.00300,1.0,0.997009,0.997009
2,2y,OIS,0.00325,2.0,NaN,NaN
3,3y,OIS,0.00335,3.0,NaN,NaN
4,4y,OIS,0.00350,4.0,NaN,NaN
5,5y,OIS,0.00360,5.0,NaN,NaN
6,7y,OIS,0.00400,7.0,NaN,NaN
7,10y,OIS,0.00450,10.0,NaN,NaN
8,15y,OIS,0.00500,15.0,NaN,NaN
9,20y,OIS,0.00525,20.0,NaN,NaN


## Solve for 2y to 5y

In [4]:
ois_rates.at[2-1,'cumsum_df']

0.9970089730807578

In [5]:
def par_swap_2y(df, ois_rate, cumsum_df):
    """
    Return PV of payer swap i.e. PV of fixed leg minus PV of floating leg
    ONLY FOR YEARS WITH NO GAPS
    paramters:
        df: discount factor a year
        ois_rate: rate for the same year
        cumsum_df: cumulative sum of df up till the year earlier
    Assume annual payout since day count fraction is not included
    """
    sum_df = cumsum_df + df
    return ois_rate*sum_df - (1 - df)
    
guess = 0.99

for i in range(2,6):
    DF = fsolve(par_swap_2y, 
                guess, 
                args=(ois_rates.at[i,'Rate'], 
                      ois_rates.at[i-1,'cumsum_df']))[0]
    ois_rates.at[i,'df'] = DF
    ois_rates.cumsum_df = ois_rates.df.cumsum()

ois_rates

,Tenor,Product,Rate,years,cumsum_df,df
0,6m,OIS,0.00250,0.5,NaN,NaN
1,1y,OIS,0.00300,1.0,0.997009,0.997009
2,2y,OIS,0.00325,2.0,1.990540,0.993531
3,3y,OIS,0.00335,3.0,2.980555,0.990015
4,4y,OIS,0.00350,4.0,3.966672,0.986117
5,5y,OIS,0.00360,5.0,4.948856,0.982184
6,7y,OIS,0.00400,7.0,NaN,NaN
7,10y,OIS,0.00450,10.0,NaN,NaN
8,15y,OIS,0.00500,15.0,NaN,NaN
9,20y,OIS,0.00525,20.0,NaN,NaN


# Solve for years with gaps
The sum of discount factors is known since the preceding years have OIS rates for equation (1) to be solved. However for some e.g. 7y there is a gap of year 6. In this case assume that sequendce of df from year 5 to 7 is linear. 
$$
\sum_{k=i}^{N+i} D(0, T_i) = \frac{D(0,T_i) + D(0,T_{i+N})}{2} \times (N+1)
$$
In that case the cumulative sum still can be calculated by using the average of the latest and computed years.

In [6]:
def par_swap_7y(df, i, ois_rates):
    """
    Return PV of payer swap i.e. PV of fixed leg minus PV of floating leg
    ONLY FOR YEARS WITH NO GAPS
    paramters:
        df:         discount factor a year
        i:          pointer of in ois_rate DataFrame
        ois_rate:   rate for the same year
    Assume annual payout since day count fraction is not included
    """
    known_sum = ois_rates.at[i-1, 'cumsum_df']

    N = ois_rates.at[i, 'years'] - ois_rates.at[i-1, 'years']
    ave_df = (df + ois_rates.at[i-1,'df'])/2   
    interp_sum = (N+1) * ave_df - ois_rates.at[i-1,'df']     # less the earliest term since it is in known_sum

    ois_rate = ois_rates.at[i, 'Rate']
    
    return (known_sum + interp_sum) * ois_rate - (1-df)

for i in range(6,len(ois_rates)):
    # solve for gapped df
    DF = fsolve(par_swap_7y,
                guess, 
                args=(i,ois_rates))[0]
    # get the average df and cumulative df over gap
    N = ois_rates.at[i, 'years'] - ois_rates.at[i-1, 'years']
    ave_df = (DF + ois_rates.at[i-1,'df'])/2   
    interp_sum = (N+1) * ave_df - ois_rates.at[i-1,'df']
    # assign df and cumsum_df values for this gap year
    ois_rates.at[i,'df'] = DF
    ois_rates.at[i,'cumsum_df'] = ois_rates.at[i-1,'cumsum_df'] + interp_sum 

ois_rates

,Tenor,Product,Rate,years,cumsum_df,df
0,6m,OIS,0.00250,0.5,NaN,NaN
1,1y,OIS,0.00300,1.0,0.997009,0.997009
2,2y,OIS,0.00325,2.0,1.990540,0.993531
3,3y,OIS,0.00335,3.0,2.980555,0.990015
4,4y,OIS,0.00350,4.0,3.966672,0.986117
5,5y,OIS,0.00360,5.0,4.948856,0.982184
6,7y,OIS,0.00400,7.0,6.898556,0.972406
7,10y,OIS,0.00450,10.0,9.782916,0.955977
8,15y,OIS,0.00500,15.0,14.477704,0.927611
9,20y,OIS,0.00525,20.0,19.033155,0.900076


In [7]:
pd.DataFrame.interpolate?

Signature:
pd.DataFrame.interpolate(
    self,
    method: 'InterpolateOptions' = 'linear',
    *,
    axis: 'Axis' = 0,
    limit: 'int | None' = None,
    inplace: 'bool_t' = False,
    limit_direction: "Literal['forward', 'backward', 'both'] | None" = None,
    limit_area: "Literal['inside', 'outside'] | None" = None,
    downcast: "Literal['infer'] | None | lib.NoDefault" = <no_default>,
    **kwargs,
) -> 'Self | None'
Docstring:
Fill NaN values using an interpolation method.

Please note that only ``method='linear'`` is supported for
DataFrame/Series with a MultiIndex.

Parameters
----------
method : str, default 'linear'
    Interpolation technique to use. One of:

    * 'linear': Ignore the index and treat the values as equally
      spaced. This is the only method supported on MultiIndexes.
    * 'time': Works on daily and higher resolution data to interpolate
      given length of interval.
    * 'index', 'values': use the actual numerical values of the index.
    * 'pad': Fi

In [9]:
# Generate the full sequence
full_sequence = np.arange(0.5, 30.5, 0.5)
# Convert 'years' column of your DataFrame to a list
existing_years = ois_rates['years'].tolist()
# Remove the existing years from the full sequence
new_years = [year for year in full_sequence if year not in existing_years]

additional_years = pd.DataFrame({
    'years': new_years,
    'Rate': [np.nan]*len(new_years),
    'df': [np.nan]*len(new_years)
})
ois_rates = pd.concat([additional_years, ois_rates])
ois_rates = ois_rates.sort_values(by='years')
ois_rates['df'] = ois_rates['df'].interpolate(method='linear')
ois_rates['Product'] = "OIS"
ois_rates['cumsum_df'] = ois_rates['df'].cumsum()

ois_rates

,years,Rate,df,Tenor,Product,cumsum_df
0,0.5,0.00250,NaN,6m,OIS,NaN
1,1.0,0.00300,0.997009,1y,OIS,0.997009
0,1.5,NaN,0.995270,NaN,OIS,1.992279
2,2.0,0.00325,0.993531,2y,OIS,2.985810
1,2.5,NaN,0.991773,NaN,OIS,3.977583
3,3.0,0.00335,0.990015,3y,OIS,4.967598
2,3.5,NaN,0.988066,NaN,OIS,5.955664
4,4.0,0.00350,0.986117,4y,OIS,6.941780
3,4.5,NaN,0.984150,NaN,OIS,7.925931
5,5.0,0.00360,0.982184,5y,OIS,8.908115
